# Introduction

<center><h3>**Welcome to the Language modeling Notebook.**</h3></center>

In this assignment, you are going to train a neural network to **generate news headlines**.
To reduce computational needs, we have reduced it to headlines about technology, and a handful of Tech giants.
In this assignment you will:
- Learn to preprocess raw text so it can be fed into an LSTM.
- Make use of the LSTM library of Pytorch, to train a Language model to generate headlines
- Use your network to generate headlines, and judge which headlines are likely or not




**What is a language model?**

Language modeling is the task of assigning a probability to sentences in a language. Besides assigning a probability to each sequence of words, the language models also assigns a probability for the likelihood of a given word (or a sequence of words) to follow a sequence of words.
— Page 105, __[Neural Network Methods in Natural Language Processing](https://www.amazon.com/Language-Processing-Synthesis-Lectures-Technologies/dp/1627052984/)__, 2017.

In terms of neural network, we are training a neural network to produce probabilities (classification) over a fixed vocabulary of words.
Concretely, we are training a neural network to produce:
$$ P ( w_{i+1} | w_1, w_2, w_3, ..., w_i), \forall i \in (1,n)$$

** Why is language modeling important? **

Language modeling is a core problem in NLP.

Language models can either be used as a stand-alone to produce new text that matches the distribution of text the model is trained on, but can also be used at the front-end of a more sophisticated model to produce better results.

Recently for example, the __[BERT](https://arxiv.org/abs/1810.04805)__ paper show-cased that pretraining a large neural network on a language modeling task can help improve state-of-the-art on many NLP tasks. 

How good can the generation of a Language model be?

If you have not seen the post about GPT-2 by OpenAI, you should read some of the samples they generated from their language model __[here](https://blog.openai.com/better-language-models/#sample1)__.
Because of computational restrictions, we will not achieve as good text production, but the same algorithm is at the core. They just use more data and compute.

# Library imports

Before starting, make sure you have all these libraries.

In [1]:
!pip install segtok

  Created wheel for segtok: filename=segtok-1.5.10-cp37-none-any.whl size=25019 sha256=bb356f897c938cd3d3d3be7a340085d7287bda4d035abad351f3ed48141b1021
  Stored in directory: /root/.cache/pip/wheels/b4/39/f6/9ca1c5cabde964d728023b5751c3a206a5c8cc40252321fb6b
Successfully built segtok


Run the first of the following two cells if you are running the homework locally, and run the second cell if you are running the homework in Colab

In [ ]:
DRIVE=False
root_folder = ""
dataset_folder = "dataset/"

In [2]:
from google.colab import drive
drive.mount('/content/drive')
root_folder = "/content/drive/My Drive/cs182_hw3/"
dataset_folder = "/content/drive/My Drive/cs182_hw3_public/dataset/"

Mounted at /content/drive


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import sys
sys.path.append(root_folder)

from segtok import tokenizer
from collections import Counter
import torch as th
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import json
from utils import validate_to_array

# Loading the datasets

Make sure the dataset files are all in the `dataset` folder of the assignment.

 - If you are using this notebook locally: You should run the `download_data.sh` script.
 - If you are using the Colab version of the notebook, make sure that your Google Drive is mounted, and you verify from the file explorer in Colab that the files are viewable within `/content/drive/cs182_hw3_public/dataset/`
 


In [5]:
# This cell loads the data for the model
# Run this before working on loading any of the additional data

with open(dataset_folder+"headline_generation_dataset_processed.json", "r") as f:
    d_released = json.load(f)

with open(dataset_folder+"headline_generation_vocabulary.txt", "r",encoding='utf8') as f:
    vocabulary = f.read().split("\n")
w2i = {w: i for i, w in enumerate(vocabulary)} # Word to index
i2w = {i: w for i, w in enumerate(vocabulary)} # Index to word
unkI, padI, startI = w2i['UNK'], w2i['PAD'], w2i['<START>']

vocab_size = len(vocabulary)
input_length = len(d_released[0]['numerized']) # The length of the first element in the dataset, they are all of the same length
d_train = [d for d in d_released if d['cut'] == 'training']
d_valid = [d for d in d_released if d['cut'] == 'validation']

print("Number of training samples:",len(d_train))
print("Number of validation samples:",len(d_valid))

Number of training samples: 88568
Number of validation samples: 946


Now that we have loaded the data, let's inspect one of the elements. Each sample in our dataset has a `numerized` vector, that contains the preprocessed headline. This vector is what we will feed in to the neural network. The field `numerized` corresponds to this list of tokens. The already loaded dictionary `vocabulary` maps token lists to the actual string. Use these elements to recover `title` key of entry 1001 in the training dataset.

**TODO**: Write the numerized2text function in notebook_utils and inspect element 1001 in the training dataset (`entry = d_train[1001]`).



In [6]:
def numerize_sequence(tokenized):
    return [w2i.get(w, unkI) for w in tokenized]
def pad_sequence(numerized, pad_index, to_length):
    pad = numerized[:to_length]
    padded = pad + [pad_index] * (to_length - len(pad))
    mask = [w != pad_index for w in padded]
    return padded, mask

In [70]:
def numerized2text(numerized):
    """ Converts an integer sequence in the vocabulary into a string corresponding to the title.
    
        Arguments:
            numerized: List[int]  -- The list of vocabulary indices corresponding to the string
        Returns:
            title: str -- The string corresponding to the numerized input, without padding.
    """
    #####
    # BEGIN YOUR CODE HERE 
    # Recover each word from the vocabulary in the list of indices in numerized, using the vocabulary variable
    # Hint 1: Use the string.join() function to reconstruct a single string
    # Hint 2: The objects and/or functions defined in above cells may be useful.
    #####
    words = [i2w.get(n) for n in numerized]
    if "PAD" in words:
      pad_index = words.index("PAD")
      words = words[:pad_index]
    converted_string = " ".join(words)
    #####
    # END YOUR CODE HERE
    #####
    
    return converted_string

entry = d_train[1001]
print("Reversing the numerized: "+numerized2text(entry['numerized']))
validate_to_array(numerized2text,(entry['numerized'],),'numerized2text',root_folder)
print("From the `title` entry: "+ entry['title'])

Reversing the numerized: microsoft donates cloud computing ' worth $ 1 bn '
From the `title` entry: Microsoft donates cloud computing 'worth $1 bn'


In language modeling, we train a model to produce the next word in the sequence given all previously generated words. This has, in practice, two steps:


    1. Adding a special <START> token to the start of the sequence for the input. This "shifts" the input to the right by one. We call this the "source" sequence
    2. Making the network predict the original, unshifted version (we call this the "target" sequence)

    
Let's take an example. Say we want to train the network on the sentence: "The cat is great."
The input to the network will be "`<START>` The cat is great." The target will be: "The cat is great".
    
Therefore the first prediction is to select the word "The" given the `<START>` token.
The second prediction is to produce the word "cat" given the two tokens "`<START>` The".
At each step, the network learns to predict the next word, given all previous ones.
    
---

Your next step is to write the build_batch function. Given a dataset, we select a random subset of samples, and will build the "inputs" and the "targets" of the batch, following the procedure we've described.

**TODO**: write the build_batch function. We give you the structure, and you have to fill in where we have left things `your_code`.


In [8]:
def build_batch(dataset, indices):
    """ Builds a batch of source and target elements from the dataset.
    
        Arguments:
            dataset: List[db_element] -- A list of dataset elements
            indices: List[int] -- A list of indices of the dataset to sample
        Returns:
            batch_input: List[List[int]] -- List of source sequences
            batch_target: List[List[int]] -- List of target sequences
            batch_target_mask: List[List[int]] -- List of target batch masks
    """
    #####
    # BEGIN YOUR CODE HERE 
    #####
    
    
    # We get a list of indices we will choose from the dataset.
    # indices = range(iteration*batch_size,(iteration+1)*batch_size)
    
    # Recover what the entries for the batch are
    batch = [dataset[i] for i in indices]
    
    # Get the raw numerized for this input, each element of the dataset has a 'numerized' key
    batch_numerized = np.array([sample['numerized'] for sample in batch])
    # Create an array of startI that will be concatenated at position 1 for the input.
    # Should be of shape (batch_size, 1)
    start_tokens = np.array([startI] * len(batch))
    start_tokens = start_tokens.reshape((-1, 1))

    # Concatenate the start_tokens with the rest of the input
    # The np.concatenate function should be useful
    # The output should now be [batch_size, sequence_length+1]
    batch_input = np.concatenate((start_tokens, batch_numerized), axis=1)
    # Remove the last word from each element in the batch
    # To restore the [batch_size, sequence_length] size
    batch_input = batch_input[:,:-1]
    
    # The target should be the un-shifted numerized input
    batch_target = batch_numerized

    # The target-mask is a 0 or 1 filter to note which tokens are
    # padding or not, to give the loss, so the model doesn't get rewarded for
    # predicting PAD tokens.
    batch_target_mask = np.array([a['mask'] for a in batch])
    #####
    # END YOUR CODE HERE 
    #####
        
    return batch_input, batch_target, batch_target_mask
validate_to_array(build_batch,(d_train, range(100)),'build_batch',root_folder)

# Creating the language model

Now that we've written the data pipelining, we are ready to write the Neural network.

The steps to setting up a neural network to do Language modeling are:
- Creating the placeholders for the model, where we can feed in our inputs and targets.
- Creating an RNN of our choice, size, and with optional parameters
- Using the RNN on our placeholder inputs.
- Getting the output from the RNN, and projecting it into a vocabulary sized dimension, so that we can make word predictions.
- Setting up the loss on the outputs so that the network learns to produce the correct words.
- Finally, choosing an optimizer, and defining a training operation: using the optimizer to minimize the loss.

We provide skeleton code for the model, you can fill in the `your_code` section. If you are unfamiliar with Pytorch, we provide some idea of what functions to look for, you should use the Pytorch online documentation.

**TODO**: Fill in the LanguageModel in the language_model file.


In [9]:
from language_model import LanguageModel

# Training the model

Your objective is to train the Language on the dataset you are provided to reach a **validation loss <= 5.50**

**TODO**: Train your model so that it achieves a validation loss of <= 5.5. 

**Careful**: we will be testing this loss on an unreleased test set, so make sure to evaluate properly on a validation set and not overfit. You must save the model you want us to test under: models/final_language_model (the .index, .meta and .data files)

**Advice**:
- It should be possible to attain loss <= 5.50 with a 1-layer LSTM of size 256 or less.
- You should not need more than 10 epochs to attain the threshold. More passes over the data can however give you a better model.
- You can however try using:
    - LSTM dropout (Pytorch has a layer for that)
    - Multi-layer RNN cell (Pytorch has a layer for that)
    - Change your optimizers, tune your learning_rate, use a learning rate schedule.
    
**Extra credit**:

Get the loss below **validation loss <= 5.00** and get 5 points of extra-credit on this assignment. Get creative,

but remember, what you do should work on our held-out test set to get the points.

In [76]:
# We can create our model,
# with parameters of our choosing.
hidden_size = 256
num_layers = 1
dropout = .5

# Setup the loss using cross-entropy loss.
# The logits are the output_logits we've computed,
# look at the pytorch docs for `CrossEntropyLoss` and `permute`
# to align the axes correctly and to account for the masking properly.
# The targets are the goal labels we are trying to match.
# Note that if you directly take the mean of the loss tensor,
# it will underestimate your loss! (why would that be?)
# Lastly, there are a few valid forms of averaging token losses,
# here we will take the mean of all non-mask tokens together.
criterion = nn.CrossEntropyLoss(reduction='none')
def loss_fn(pred, target, mask):
    pred = pred.permute(0,2,1)  # put the class probabilities in the middle
    loss_tensor = criterion(pred, target)
    loss_masked = th.mul(loss_tensor, mask)
    loss_per_sample = loss_masked.sum() / mask.sum()
    return loss_per_sample

# The build_batch function outputs numpy, but our model is built in pytorch,
# so you need to convert numpy to pytorch.
# You also have to cast the masks into float32, target into long, and input into long.
# Look at the `float` and `long` function.
batch_to_torch = lambda b_in,b_target,b_mask: (th.LongTensor(b_in),
                                               th.LongTensor(b_target), 
                                               th.FloatTensor(b_mask))


# Look at the docs for torch.optim and pick an optimizer
# And provide it with a start learning rate.
optimizer_class = optim.Adam
lr = 1e-4
epochs = 20
batch_size = 128

model_id = 'test1'
os.makedirs(root_folder+'models/part1/',exist_ok=True)

device = th.device("cuda" if th.cuda.is_available() else "cpu")
print(device)
list_to_device = lambda th_obj: [tensor.to(device) for tensor in th_obj]

cuda


In [87]:
model = LanguageModel(vocab_size=vocab_size, rnn_size=hidden_size, num_layers=num_layers, dropout=dropout)
optimizer = optimizer_class(model.parameters(), lr=lr)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [95]:
# Skeleton code
# You have to write your own training process to obtain a
# Good performing model on the validation set, and save it.

model.train()
losses = []
accuracies = []
from tqdm import tqdm
for epoch in range(epochs):
    indices = np.random.permutation(range(len(d_train)))
    t = tqdm(range(0,(len(d_train)//batch_size)+1))
    for i in t:
        # Here is how you obtain a batch:
        batch = build_batch(d_train, indices[i*batch_size:(i+1)*batch_size])
        (batch_input, batch_target, batch_target_mask) = batch_to_torch(*batch)
        (batch_input, batch_target, batch_target_mask) = list_to_device((batch_input, batch_target, batch_target_mask))
        model.to(device)
        
        prediction = model.forward(batch_input)
        loss = loss_fn(prediction, batch_target, batch_target_mask)
        losses.append(loss.item())
        accuracy = (th.eq(prediction.argmax(dim=2,keepdim=False),batch_target).float()*batch_target_mask).sum()/batch_target_mask.sum()
        accuracies.append(accuracy.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            t.set_description(f"Epoch: {epoch} Iteration: {i} Loss: {np.mean(losses[-10:])} Accuracy: {np.mean(accuracies[-10:])}")
    # save your latest model
    save_dict = dict(
        kwargs = dict(
            vocab_size=vocab_size,
            rnn_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout
        ),
        model_state_dict = model.state_dict(),
        notes = "",
        optimizer_class = optimizer_class,
        lr = lr,
        epochs = epochs,
        batch_size = batch_size,
    )
    th.save(save_dict,root_folder+f'models/part1/model_{model_id}.pt')

Epoch: 0 Iteration: 690 Loss: 5.106510925292969 Accuracy: 0.17083151638507843: 100%|██████████| 692/692 [00:17<00:00, 38.47it/s]
Epoch: 1 Iteration: 690 Loss: 5.1345030784606935 Accuracy: 0.17368502169847488: 100%|██████████| 692/692 [00:18<00:00, 37.91it/s]
Epoch: 2 Iteration: 690 Loss: 5.114482879638672 Accuracy: 0.1720510795712471: 100%|██████████| 692/692 [00:18<00:00, 37.81it/s]
Epoch: 3 Iteration: 690 Loss: 5.126867866516113 Accuracy: 0.16970811635255814: 100%|██████████| 692/692 [00:17<00:00, 38.46it/s]
Epoch: 4 Iteration: 690 Loss: 5.141033792495728 Accuracy: 0.17163413316011428: 100%|██████████| 692/692 [00:17<00:00, 38.60it/s]
Epoch: 5 Iteration: 690 Loss: 5.125402212142944 Accuracy: 0.17187462896108627: 100%|██████████| 692/692 [00:18<00:00, 38.34it/s]
Epoch: 6 Iteration: 690 Loss: 5.1453639507293705 Accuracy: 0.16728989481925965: 100%|██████████| 692/692 [00:18<00:00, 38.18it/s]
Epoch: 7 Iteration: 690 Loss: 5.129109573364258 Accuracy: 0.1654505431652069: 100%|██████████| 6

# Using the language model

Congratulations, you have now trained a language model! We can now use it to evaluate likely news headlines, as well as generate our very own headlines.

**TODO**: Complete the three parts below, using the model you have trained.

## (1) Evaluation loss

To evaluate the language model, we evaluate its loss (ability to predict) on unseen data that is reserved for evaluation.
Your first evaluation is to load the model you trained, and obtain a test loss. If you are running this validation and not training, run the setup cell above the training loop first.

In [96]:
model_id = "test1"
save_dict = th.load(root_folder+'models/part1/'+f"model_{model_id}.pt",map_location='cpu')
model = LanguageModel(**save_dict['kwargs'])
model.load_state_dict(save_dict['model_state_dict'])
model.eval()

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


LanguageModel(
  (embedding): Embedding(10000, 256)
  (lstm): LSTM(256, 256, batch_first=True, dropout=0.5)
  (output): Linear(in_features=256, out_features=10000, bias=True)
)

In [97]:
# We will evaluate your model in the best_models folder
# In a very similar way as the code below.
# Make sure your validation loss is below the threshold we specified
# and that you didn't train using the validation set

batch = build_batch(d_valid, range(len(d_valid)))
(batch_input, batch_target, batch_target_mask) = batch_to_torch(*batch)
prediction = model(batch_input.long())
loss = loss_fn(prediction, batch_target, batch_target_mask)
print("Evaluation set loss:", loss.item())

Evaluation set loss: 5.4604620933532715


In [98]:
# Your best performing model should go here.
os.makedirs(root_folder+"best_models",exist_ok=True)
best_model_file = root_folder+"best_models/part1_best_model.pt"
th.save(save_dict,best_model_file)

## (2) Evaluation of likelihood of data

One use of a language model is to see what data is more likely to have originated from the training data. Because we have trained our model on news headlines, we can see which of these headlines is more likely:

``Apple to release another iPhone in September``


 ``Apple and Samsung resolve all lawsuits amicably``
 
**TODO**: Use the model to obtain the loss the neural network assigns to each sentence.
Because the neural network assigns probability to the words appearing in a sequence, this loss can be used as a proxy to measure how likely the sentence is to have occurred in the dataset.
Once you have the loss for each headline, write down which sentence was judged to be more likely, and explain why/if you think this is coherent.

**Your answer:**


In [99]:
def raw_sample_pred(headline, model):
    #####
    # BEGIN YOUR CODE HERE 
    #####
    # From the code in the Preprocessing section at the end of the notebook
    # Find out how to tokenize the headline
    tokenized = tokenizer.word_tokenizer(headline.lower())

    # Find out how to numerize the tokenized headline
    numerized = numerize_sequence(tokenized)

    # Learn how to pad and obtain the mask of the sequence.
    padded, mask = pad_sequence(numerized, padI, input_length)

    # Obtain the predicted headline and target headline
    input_headline = [startI] + padded
    input_headline = th.LongTensor(input_headline[:-1])
    input_headline = th.reshape(input_headline, (1, -1))
    pred_headline = model.forward(input_headline)
    target_headline = th.LongTensor(padded)
    target_headline = th.reshape(target_headline, (1, -1))
    mask = th.FloatTensor(mask)
    mask = th.reshape(mask, (1, -1))

    #####
    # END YOUR CODE HERE 
    #####

    return pred_headline,target_headline,mask

In [100]:
model.eval()

headline1 = "Apple to release new iPhone in July"
headline2 = "Apple and Samsung resolve all lawsuits"

headlines = [headline1.lower(), headline2.lower()] # Our LSTM is trained on lower-cased headlines
for headline in headlines:
    pred_headline,target_headline,mask = raw_sample_pred(headline, model)
    loss = loss_fn(pred_headline, target_headline, mask) # Obtain the loss
    
    print("----------------------------------------")
    print("Headline:", headline)
    print("Loss of the headline:", loss)
validate_to_array(raw_sample_pred,zip(headlines,[model]*2),'raw_sample_pred',root_folder,multi=True)
# Important check: one headline should be more likely (and have lower loss)
# Than the other headline. You should know which headline should have lower loss.

----------------------------------------
Headline: apple to release new iphone in july
Loss of the headline: tensor(3.4249, grad_fn=<DivBackward0>)
----------------------------------------
Headline: apple and samsung resolve all lawsuits
Loss of the headline: tensor(5.3695, grad_fn=<DivBackward0>)


## (3) Generation of headlines

We can use our language model to generate text according to the distribution of our training data.
The way generation works is the following:

We seed the model with a beginning of sequence, and obtain the distribution for the next word.
We select the most likely word (argmax) and add it to our sequence of words.
Now our sequence is one word longer, and we can feed it in again as an input, for the network to produce the next sentence.
We do this a fixed number of times (up to 20 words), and obtain automatically generated headlines!


We have provided a few headline starters that should produce interesting generated headlines.

**TODO:** Get creative and find at least 2 more headline_starters that produce interesting headlines.

In [101]:
def generate_sentence(headline_starter, model):
    # Tokenize and numerize the headline. Put the numerized headline
    # beginning in `current_build`
    tokenized = tokenizer.word_tokenizer(headline_starter.lower())
    current_build = [startI] + numerize_sequence(tokenized)

    while len(current_build) < input_length:
        # Pad the current_build into a input_length vector.
        # We do this so that it can be processed by our LanguageModel class
        current_padded, _m = pad_sequence(current_build, padI, input_length)

        # Obtain the logits for the current padded sequence
        # This involves obtaining the output_logits from our model,
        # and not the loss like we have done so far
        input_sequence = th.LongTensor(current_padded)
        input_sequence = th.reshape(input_sequence, (1, -1))
        logits = model.forward(input_sequence)
        logits_np = logits.detach().cpu().numpy()

        # Obtain the row of logits that interest us, the logits for the last non-pad
        # inputs
        last_logits = logits_np[0, len(current_build), :]

        # Find the highest scoring words in the last_logits
        # array, or sample from the softmax.
        # The np.argmax function may be useful for first option,
        # sp.special.softmax and np.random.choice may be useful for second option.
        # Append this word to our current build
        current_build.append(np.argmax(last_logits))

    # Go from the current_build of word_indices
    # To the headline (string) produced. This should involve
    # the vocabulary, and a string merger.
    produced_sentence = numerized2text(current_build)
    return produced_sentence

In [102]:
model.eval()
# Here are some headline starters.
# They're all about tech companies, because
# That is what is in our dataset
headline_starters = ["apple has released", "google has released", "amazon", "tesla to"]
for headline_starter in headline_starters:
    print("===================")
    print("Generating headline starting with: "+headline_starter)

    produced_sentence = generate_sentence(headline_starter, model)
    print(produced_sentence)
validate_to_array(generate_sentence,zip(headline_starters,[model]*len(headline_starters)),"generate_sentence",root_folder,multi=True)

Generating headline starting with: apple has released
<START> apple has released a in , UNK on , , , , , , , , , , ,
Generating headline starting with: google has released
<START> google has released in with of in , , , , , , , , , , , ,
Generating headline starting with: amazon
<START> amazon UNK in , , , , , , , , , , , , , , , ,
Generating headline starting with: tesla to
<START> tesla to $ billion in , , , , , , , , , , , , , ,


## All done

You are done with the first part of the HW.

Next notebook deals with Summarization of text!


# Preprocessing (read only)


**You can skip this section, however you may find these functions useful in later sections of this notebook**

We have provided this code so you see how the dataset was generated. You will have to come back some of these functions later in the assignment, so feel free to read through, to get familiar.

In [16]:
# You do not need to run this
# This is to show you how the dataset was created
# You should read to understand, so you can preprocess text
# In the same way, in the evaluation section

for a in dataset:
    a['tokenized'] = tokenizer.word_tokenizer(a['title'].lower())

NameError: ignored

In [ ]:
# You do not need to run this
# This is to show you how the dataset was created
# You should read to understand, so you can preprocess text
# In the same way, in the evaluation section

word_counts = Counter()
for a in dataset:
    word_counts.update(a['tokenized'])

print(word_counts.most_common(30))

In [ ]:
# You do not need to run this
# This is to show you how the dataset was created
# You should read to understand, so you can preprocess text
# In the same way, in the evaluation section

# Creating the vocab
vocab_size = 20000
special_words = ["<START>", "UNK", "PAD"]
vocabulary = special_words + [w for w, c in word_counts.most_common(vocab_size-len(special_words))]
w2i = {w: i for i, w in enumerate(vocabulary)}

# Numerizing and padding
input_length = 20
unkI, padI, startI = w2i['UNK'], w2i['PAD'], w2i['<START>']

for a in dataset:
    a['numerized'] = numerize_sequence(a['tokenized']) # Change words to IDs
    a['numerized'], a['mask'] = pad_sequence(a['numerized'], padI, input_length) # Append appropriate PAD tokens
    
# Compute fraction of words that are UNK:
word_counters = Counter([w for a in dataset for w in a['input'] if w != padI])

print("Fraction of UNK words:", float(word_counters[unkI]) / sum(word_counters.values()))

In [ ]:
# You do not need to run this
# This is to show you how the dataset was created
# You should read to understand, so you can preprocess text
# In the same way, in the evaluation section

d_released_processed   = [d for d in dataset if d['cut'] != 'testing']
d_unreleased_processed = [d for d in dataset if d['cut'] == 'testing']

with open("dataset/headline_generation_dataset_processed.json", "w") as f:
    json.dump(d_released_processed, f)

# This file is purposefully left out of the assignment, we will use it to evaluate your model.
with open("dataset/headline_generation_dataset_unreleased_processed.json", "w") as f:
    json.dump(d_unreleased_processed, f)
    
with open("dataset/headline_generation_vocabulary.txt", "w") as f:
    f.write("\n".join(vocabulary).encode('utf8'))